* https://arxiv.org/abs/1408.5882
* http://docs.likejazz.com/cnn-text-classification-tf/
* https://github.com/Shawn1993/cnn-text-classification-pytorch

In [49]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import json
import pickle
import random
import time
import math
import numpy as np
from konlpy.tag import Mecab;tagger=Mecab()
from collections import Counter
from torch.nn.utils.rnn import PackedSequence,pad_packed_sequence, pack_padded_sequence

torch.manual_seed(1)

In [3]:
USE_CUDA = torch.cuda.is_available()

We initially keep the
word vectors static and learn only the other parameters
of the model <br>
 Learning task-specific vectors through
fine-tuning results in further improvements<br>
 We
finally describe a simple modification to the architecture
to allow for the use of both pre-trained and
task-specific vectors by having multiple channels.

In [153]:
class  CNN_Text(nn.Module):
    
    def __init__(self, embed_num,embed_dim,class_num,kernel_num,kernel_sizes,dropout):
        super(CNN_Text,self).__init__()
        #self.args = args
        
        V = embed_num # num of vocab
        D = embed_dim # dimenstion of word vector
        C = class_num # num of class
        Ci = 1
        Co = kernel_num # 100
        Ks = kernel_sizes # [3,4,5]

        self.embed = nn.Embedding(V, D)
        #self.convs1 = [nn.Conv2d(Ci, Co, (K, D)) for K in Ks]
        self.convs1 = nn.ModuleList([nn.Conv2d(Ci, Co, (K, D)) for K in Ks])
        
        # kernal_size = (K,D) : D는 단어 벡터 길이라 픽스, K 사이즈만큼 슬라이딩, 스트라이드는 1
        
        '''
        self.conv13 = nn.Conv2d(Ci, Co, (3, D))
        self.conv14 = nn.Conv2d(Ci, Co, (4, D))
        self.conv15 = nn.Conv2d(Ci, Co, (5, D))
        '''
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(len(Ks)*Co, C)

    def conv_and_pool(self, x, conv):
        x = F.relu(conv(x)).squeeze(3) #(N,Co,W)
        x = F.max_pool1d(x, x.size(2)).squeeze(2)
        return x


    def forward(self, x,train=True):
        x = self.embed(x) # (N,W,D)
        
        #if self.args.static:
        #    x = Variable(x)

        x = x.unsqueeze(1) # (N,Ci,W,D)

        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1] #[(N,Co,W), ...]*len(Ks)


        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x] #[(N,Co), ...]*len(Ks)

        x = torch.cat(x, 1)

        '''
        x1 = self.conv_and_pool(x,self.conv13) #(N,Co)
        x2 = self.conv_and_pool(x,self.conv14) #(N,Co)
        x3 = self.conv_and_pool(x,self.conv15) #(N,Co)
        x = torch.cat((x1, x2, x3), 1) # (N,len(Ks)*Co)
        '''
        if train:
            x = self.dropout(x) # (N,len(Ks)*Co)
        logit = self.fc1(x) # (N,C)
        return logit

# Data Prepare

In [282]:
import json
from konlpy.tag import Mecab
tagger = Mecab()

stopwords=['dummy']
# Helper functions to make the code more readable.
def prepare_sequence(seq, word_to_ix):
    idxs=[]
    for s in seq:
        
        if s.isdigit():
            idxs.append(word_to_ix['NUM'])
            continue
        try:    
            if s not in stopwords:
                idxs.append(word_to_ix[s])
            else:
                idxs.append(word_to_ix['UNK'])
        except:
            idxs.append(word_to_ix['UNK'])
    
    #idxs = list(map(lambda w: to_ix[w], seq))
    tensor = torch.LongTensor(idxs)
    tensor = Variable(tensor)
    if USE_CUDA: tensor = tensor.cuda()
    
    return tensor

In [102]:
train_data = json.load(open('../../dataset/corpus/domain_dump_07_10_17.json'))

In [103]:
X,y = zip(*train_data)

In [104]:
word_to_ix={'PAD':0,'UNK':1,'NUM':2}

for x in X:
    tokens = tagger.morphs(x)
    
    for token in tokens:
        if token.isnumeric():
            token = 'NUM'
        if token not in word_to_ix:
            word_to_ix[token]=len(word_to_ix)

ix_to_word = {v:k for k,v in word_to_ix.items()}

target_to_ix={}

for y_ in y:
    if y_ not in target_to_ix:
        target_to_ix[y_]=len(target_to_ix)
        
ix_to_target = {v:k for k,v in target_to_ix.items()}

In [145]:
X_p = []
except_index=[]
for x in X:
    tokens = tagger.morphs(x)
    while len(tokens) < 5:
        tokens.append('PAD')
        
    tokens = ['NUM' if token.isnumeric() else token for token in tokens]
    X_p.append(prepare_sequence(tokens,word_to_ix).view(1,-1))

In [182]:
model = CNN_Text(len(word_to_ix),30,len(target_to_ix),30,[3,4,5],0.8)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [183]:
train = list(zip(X_p,list(y)))

In [184]:
for step in range(100):
    losses=[]
    for i,(sent,target) in enumerate(train):
        
        pred = model(sent)
        target = Variable(torch.LongTensor([target_to_ix[target]]))
        loss = loss_function(pred,target)
        loss.backward()
        optimizer.step()
        losses.append(loss)
    if step % 10==0:
        print(step,np.mean(losses).data.tolist()[0])

0 1.446259617805481
10 1.254598617553711
20 1.2565611600875854
30 1.1359226703643799
40 2.089221239089966
50 2.5133934020996094
60 0.26536357402801514
70 0.24461354315280914
80 0.25541332364082336
90 1.168743371963501


In [345]:
test = ["다음주 부산 날씨 머야?",
            "오늘 저녁에 집 앞에서 미팅",
            "이거 어떻게 하는거야?",
            "꽃다발 배달하고 싶어!",
            "장미 배달 돼? 빨간색으로",
            "이거 너무하네 진짜",
            "ㅇㅇㅇㅇ내일 비 와??",
            "이번 주말에 서울에 비 안오지?",
            "하이루 내일 뭐하니",
            "조화 보내고 싶은데 어케해",
            "ㅋㅋㅋ이메일 보내고 싶어!",
            "어쭈구리?ㅋㅋ 혼난다ㅡㅡ",
           "됐고ㅡㅡ 내일 날씨나 알려줘",
           "오늘 일정이 어케됭?ㅋ",
           "흐아.. 진자 못맞추는구나 보내고",
            "어키,, 알겠으 정말 바보구나 싶어",
           "헐 날씨 넘나 좋은것",
            "문봇의 문이 뭐야?",
            "너 꽃배달 업체가 어디야?",
            "꽃만 들어 있으면 이걸로 판단하는거냐",
            "ㅋㅋㅋ너네 패턴 다 파악함",
            "날씨도 좋은데 꽃이나 보낼래",
            "꽃은 안보내도 되고, 날씨나 알려줘"]

In [346]:
for t in test:
    tokens = tagger.morphs(t)
    input = prepare_sequence(tokens,word_to_ix)
    pred = model(input.view(1,-1),False)
    v,i = torch.max(pred,1)
    result = ix_to_target[i.data.tolist()[0][0]]
    
    print(t,result)

다음주 부산 날씨 머야? WEATHER
오늘 저녁에 집 앞에서 미팅 SCHEDULE
이거 어떻게 하는거야? OTHER
꽃다발 배달하고 싶어! FLOWER
장미 배달 돼? 빨간색으로 FLOWER
이거 너무하네 진짜 OTHER
ㅇㅇㅇㅇ내일 비 와?? WEATHER
이번 주말에 서울에 비 안오지? OTHER
하이루 내일 뭐하니 OTHER
조화 보내고 싶은데 어케해 FLOWER
ㅋㅋㅋ이메일 보내고 싶어! FLOWER
어쭈구리?ㅋㅋ 혼난다ㅡㅡ OTHER
됐고ㅡㅡ 내일 날씨나 알려줘 OTHER
오늘 일정이 어케됭?ㅋ OTHER
흐아.. 진자 못맞추는구나 보내고 FLOWER
어키,, 알겠으 정말 바보구나 싶어 FLOWER
헐 날씨 넘나 좋은것 OTHER
문봇의 문이 뭐야? OTHER
너 꽃배달 업체가 어디야? FLOWER
꽃만 들어 있으면 이걸로 판단하는거냐 FLOWER
ㅋㅋㅋ너네 패턴 다 파악함 OTHER
날씨도 좋은데 꽃이나 보낼래 FLOWER
꽃은 안보내도 되고, 날씨나 알려줘 FLOWER
